In [58]:
# First we will import the sqlite3 library
import sqlite3

In Python, we can run SQL command as we did previously. But first, we need to connect to the database engine and open the database file. Then we will create the cursor object. The cursor object is where we execute SQL command.

In [59]:
# Change the following line to the path where you created the database (crop_record.db)
database_location = '/tmp/test.db'
# Connect to an SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(database_location)
# Create a cursor object using the cursor() method
cursor = conn.cursor()
# Read the data from the plots table. The return value is a list of rows.
cursor.execute("SELECT * FROM plots").fetchall()
# It is a good practice to close the connection once we are done.
conn.close()

As you see, you can manage the database through Python script. You can change `cursor.execute()` to create tables, inserts rows, or everything that you can do in the SQL commands. Please note that, if you make changes to the database (e.g. create tables, insert rows, delete or modify values), the changes are not recorded yet until you run `conn.commit()` (you need to commit before closing the database connection). 

Using Python will give yor more flexibility to work with the database. For example, if you need to inserts hundreds of row, you can use Python to generate insert commands instead of typing each command manually. You can also connect the data with other Python's ecosystem. For example, plot graph with Matplotlib, data science with Pandas, or building web application which we will do in next modules.  

However, there is a another way to work with SQL using Object Relational Mapper or ORM. ORM is a bridge between database and Object Oriented Programming. There are several ORM libraries for many programming languages. For Python, we will use the library called "SQLAlchemy" Let's see how it works in action. 

In [ ]:
# Install sqlalchemy. You only need to run this once.
%pip install sqlalchemy==2.0.*
# Automatically reload modules that have changed while running the notebook
%load_ext autoreload
%autoreload 2

In [61]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

# Create a SQLAlchemy engine to connect to the database
engine = create_engine(f'sqlite:///{database_location}')
# Create a session object to manage the database (similar to a cursor in sqlite3)
session = Session(engine)


Now we need to create classes that will represent our crops table and plots table. We will use `data_model.py` to declare our models. So open `data_model.py` and look at the `class Crop`. Then come back to this notebook. 

In [ ]:
from data_model import Crop

# Now we can query (select) the data from the crops table
session.query(Crop).all()

In [ ]:
# The advantage of using SQLAlchemy is that we can access the columns as attributes of the object.
crops = session.query(Crop).all()
for crop in crops:
    print(crop.crop_id, crop.crop_name, crop.maturity_date, crop.note)

Now we change our playground from plan SQL command to Python OOP style. With OOP (class), we can add functions or methods into the class blueprint. You can also insert new rows. Like SQL, you don't need to worry about primary key column. The engine will take care of it automatically. 

In [ ]:
new_crop = Crop(crop_name='tomato', maturity_date=90, note='Plant in rows')
session.add(new_crop)                   # Add the new crop into the database 
# session.commit()                        # Commit the transaction to save the changes
# You might want to leave the commit command commented out. Only uncomment it when you are sure you want to save the changes.
session.query(Crop).all()               # Query the database to see new row

Now, it is your turn. Write a model class for the plot table. Go back to `data_model.py` and complete the class `Plot`. Plot id, planted_date, and crop_id columns are provided. Take a look at how we can define foreign key in the crop id column. The string representation should contain, plot id, plot name, plot area, crop id, and planted date. 

In [ ]:
from data_model import Plot # Test if you defined the Plot class correctly
session.query(Plot).all()

Now you can query the plots table and show what crop was planted on that plot. However, showing crop id is not very helpful. It will be much better if you show the crop name instead. So, let's make some changes. Uncomment `plots: Mapped[List["Plot"]] = relationship(back_populates="crop")` in class `Crop` and `crop: Mapped["Crop"] = relationship(back_populates="plots")` in class `Plot`. These two lines are not actual columns in the database. But they are mapping relationship that make ORM very powerful. Now, each `Plot` object has crop attribute with is point to `Crop` object and this allow you to access `Crop` attributes. Let's upgrade class `Plot`'s `__repr__` from `{self.crop_id}` to `{self.crop.crop_name}`. 



In [ ]:
# Let's query again and see changes. 
# You may need to click Restart on the top panel if you encounter an error. Then click Run All.
session.query(Plot).all()